In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# 1. Setup Parameters
num_transactions = 500  # Number of journal entries
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

# 2. Define Account Rules (To make data realistic)
# Structure: [Debit Account, Credit Account, Min Amount, Max Amount]
patterns = [
    ['1100', '4000', 1000, 5000],  # Sale: Debit AR (Asset), Credit Revenue
    ['5000', '1200', 500, 2500],   # COGS: Debit COGS (Exp), Credit Inventory (Asset)
    ['6000', '2000', 100, 1000],   # OpEx: Debit OpEx (Exp), Credit AP (Liab)
    ['2000', '1000', 100, 1000],   # Pay Bills: Debit AP (Liab), Credit Cash (Asset)
    ['1000', '1100', 1000, 5000],  # Collect Cash: Debit Cash, Credit AR
]

data = []

print("🔄 Generating Mock Financial Data...")

for i in range(num_transactions):
    # Randomly pick a pattern
    pattern = random.choice(patterns)
    debit_acct = pattern[0]
    credit_acct = pattern[1]
    
    # Random Amount
    amount = round(random.uniform(pattern[2], pattern[3]), 2)
    
    # Random Date
    random_days = random.randint(0, (end_date - start_date).days)
    trx_date = start_date + timedelta(days=random_days)
    
    # Transaction ID
    trx_id = f"TRX-{i+1000}"
    
    # Create TWO rows for every transaction (Double Entry Accounting)
    # 1. The Debit (Positive)
    data.append([trx_id, trx_date, debit_acct, 'System Generated', amount])
    # 2. The Credit (Negative)
    data.append([trx_id, trx_date, credit_acct, 'System Generated', -amount])

# 3. Create DataFrame
df_gl = pd.DataFrame(data, columns=['TransactionID', 'TransactionDate', 'AccountCode', 'Description', 'AmountGHS'])

# 4. Preview
print(f"✅ Generated {len(df_gl)} rows.")
df_gl.head(10)

🔄 Generating Mock Financial Data...
✅ Generated 1000 rows.


,TransactionID,TransactionDate,AccountCode,Description,AmountGHS
0,TRX-1000,2024-06-07,5000,System Generated,1783.54
1,TRX-1000,2024-06-07,1200,System Generated,-1783.54
2,TRX-1001,2024-05-29,1000,System Generated,3834.48
3,TRX-1001,2024-05-29,1100,System Generated,-3834.48
4,TRX-1002,2024-10-19,2000,System Generated,147.67
5,TRX-1002,2024-10-19,1000,System Generated,-147.67
6,TRX-1003,2024-08-02,1100,System Generated,4978.93
7,TRX-1003,2024-08-02,4000,System Generated,-4978.93
8,TRX-1004,2024-02-13,1000,System Generated,1622.90
9,TRX-1004,2024-02-13,1100,System Generated,-1622.90


In [2]:
import sys
import os
import sqlalchemy

# 1. Add the 'src' folder to Python's path so we can find 'utils'
current_dir = os.getcwd() # Gets the folder where this notebook is
project_root = os.path.dirname(os.path.dirname(current_dir)) # Goes up 2 levels
sys.path.append(project_root)

# 2. Import your connection tool
from src.utils.db_connect import get_db_connection

# 3. Connect
engine = get_db_connection()

if engine:
    print("🚀 Connected to MSSQL. Writing data...")
    
    # 4. Write DataFrame to SQL
    # 'if_exists="append"' adds this data to your table. 
    # We use chunksize to prevent crashing if you have millions of rows.
    try:
        df_gl.to_sql('Fact_GeneralLedger', con=engine, if_exists='append', index=False, chunksize=1000)
        print(f"✅ Success: {len(df_gl)} transactions loaded into Fact_GeneralLedger.")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Could not connect to database.")

✅ Successfully connected to: GILB-PREDATOR -> Financial_DB
🚀 Connected to MSSQL. Writing data...
✅ Success: 1000 transactions loaded into Fact_GeneralLedger.
